# 2024 생명연구자원 AI활용 경진대회 : 인공지능 활용 부문
#### https://dacon.io/competitions/official/236355/overview/description
#### LLM 이용 Classification
#### 사전학습모델: dmis-lab/biobert-v1.1

In [3]:
import pandas as pd
import numpy as np
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
# 결과를 저장할 리스트
data = []

# 각 row를 순회하면서 'WT'가 아닌 변이들만 추출
for index, row in train.iterrows():
    subclass = row['SUBCLASS']
    mutations = []

    # 'ID'와 'SUBCLASS'를 제외한 나머지 컬럼에서 'WT'가 아닌 값을 추출
    for col in train.columns[2:]:
        if row[col] != "WT":  # 변이가 'WT'가 아닐 때만
            mutations.append(f"{col}:{row[col]}")

    # 변이가 있을 경우 리스트에 추가
    if mutations:
        data.append({
            "subclass": subclass,
            "mutations": " ".join(mutations)
        })

In [ ]:
# 데이터셋 변환 전처리 다시하기
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

labels = ['KIPAN', 'SARC', 'SKCM', 'KIRC', 'GBMLGG', 'STES', 'BRCA', 'THCA',
          'LIHC', 'HNSC', 'PAAD', 'OV', 'PRAD', 'UCEC', 'LAML', 'COAD',
          'ACC', 'LGG', 'LUSC', 'LUAD', 'CESC', 'PCPG', 'THYM', 'BLCA',
          'TGCT', 'DLBC']

# 데이터셋 변환 (훈련용) 
formatted_data = []
for entry in data:
    input_text = f"SUBCLASS: {entry['subclass']} [SEP] {entry['mutations']}"
    label = labels.index(entry['subclass']) if entry['subclass'] in labels else -1
    formatted_data.append({"input": input_text, "label": label, "subclass": entry['subclass']})  # 'subclass' 추가

# 데이터 섞기
random.shuffle(formatted_data)

# 80:20 비율로 데이터 나누기
split_index = int(len(formatted_data) * 0.8)
train_data = formatted_data[:split_index]
# eval_data 코드 수정 
# eval_data = [{"input": entry["input"].replace(f"SUBCLASS: {entry['subclass']}", "[UNKNOWN]"), "label": entry["label"]} for entry in formatted_data[split_index:]]

# Hugging Face Dataset으로 변환
data_train = Dataset.from_list(train_data)
data_eval = Dataset.from_list(eval_data)

In [ ]:
train_data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load BioBERT tokenizer and model for sequence classification
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=26)

In [ ]:
# 토큰화
def tokenize_function(examples):
    return tokenizer(examples["input"], padding="max_length", truncation=True)

# Tokenize the training dataset
tokenized_dataset = data_train.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Tokenize the evaluation dataset
tokenized_evaluation_dataset = data_eval.map(tokenize_function, batched=True)
tokenized_evaluation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
print("Tokenized training data sample:", tokenized_dataset[0])
print("Tokenized evaluation data sample:", tokenized_evaluation_dataset[0])


In [ ]:
# 훈련 인수 설정 (예정: f1 score 추가하기)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_evaluation_dataset,  # Set the evaluation dataset here
)

# 모델 훈련
trainer.train()

# 모델 평가
trainer.evaluate()

## test.csv 예측

In [ ]:
# 모델과 토크나이저 로드 (Fine-tuned model)
model = AutoModelForSequenceClassification.from_pretrained("/content/results/checkpoint-2444")
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

In [ ]:
# 예측할 클래스 저장할 리스트
predictions = []

# test.csv 전처리 및 예측
for _, row in test.iterrows():
    # 입력 데이터 포맷팅
    new_input = " ".join([f"{col}:{row[col]}" for col in test.columns if col != 'ID' and str(row[col]) != 'WT'])  # WT가 아닌 유전체 관련 컬럼만 결합
    formatted_input = f"SUBCLASS: [UNKNOWN] [SEP] {new_input}"  # [UNKNOWN]은 예측할 클래스가 없음

    # 입력 데이터 토크나이즈
    inputs = tokenizer(formatted_input, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

    # 모델 평가 모드로 전환
    model.eval()

    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = labels[torch.argmax(logits, dim=1).item()]

    # 예측된 클래스 저장
    predictions.append(predicted_class)
    if (_ + 1) % 100 == 0:
      print(_+1)



In [ ]:
# sample_submission.csv 파일 로드
submission_df = pd.read_csv('sample_submission.csv')

# 예측 결과를 sample_submission DataFrame에 덮어쓰기
submission_df['SUBCLASS'] = predictions

# 결과를 csv 파일로 저장
submission_df.to_csv('submission_0929.csv', index=False)